In [14]:
import pandas as pd
import pickle
import os
import glob

# Load data from the pickle file
DataPath = "./GeneratedData/*.feather"
FileSource = './GeneratedData/allData.pickle'


In [17]:
FileList=[]
with open(FileSource,'rb') as f:
    FileList = pickle.load(f)
#HeaderWithoutAccel = ["ApproachRate", "Rel_Pos_Magnitude", "SteerB", "A_Head_Center_Distance", "B_Head_Center_Distance", "Filtered_B_Head_Velocity_Total","RelativeRotation"] # Signaling ()

['./GeneratedData/CP1_ISR_03.feather',
 './GeneratedData/CP8_ISR_03.feather',
 './GeneratedData/CP7_ISR_03.feather',
 './GeneratedData/CP3_ISR_03.feather',
 './GeneratedData/CP2_ISR_03.feather',
 './GeneratedData/CP5_ISR_03.feather',
 './GeneratedData/CP6_ISR_03.feather',
 './GeneratedData/CP3_ISR_03.feather',
 './GeneratedData/CP6_ISR_03.feather',
 './GeneratedData/CP8_ISR_03.feather',
 './GeneratedData/CP7_ISR_03.feather',
 './GeneratedData/CP7_ISR_03.feather',
 './GeneratedData/CP3_ISR_03.feather',
 './GeneratedData/CP1_ISR_03.feather',
 './GeneratedData/CP6_ISR_03.feather',
 './GeneratedData/CP2_ISR_03.feather',
 './GeneratedData/CP8_ISR_03.feather',
 './GeneratedData/CP5_ISR_03.feather',
 './GeneratedData/CP7_ISR_03.feather',
 './GeneratedData/CP3_ISR_03.feather',
 './GeneratedData/CP5_ISR_03.feather',
 './GeneratedData/CP1_ISR_03.feather',
 './GeneratedData/CP2_ISR_03.feather',
 './GeneratedData/CP8_ISR_03.feather',
 './GeneratedData/CP6_ISR_03.feather',
 './GeneratedData/CP8_ISR

In [25]:
# Combine all feather files into a single DataFrame
all_data = []

for file in FileList:
    df = pd.read_feather(file)
    scenario = os.path.basename(file).split('_')[0]
    location = os.path.basename(file).split('_')[1]
    run = os.path.basename(file).split('_')[2].split('.')[0]
    df['Scenario'] = scenario
    df['Location'] = location
    df['Run'] = run 
    all_data.append(df)


In [26]:

# Combine all data into a single DataFrame
data = pd.concat(all_data, ignore_index=True)

In [28]:
# Example: ANOVA test for one feature
anova_results = {}

for feature in ['A_Head_DeltaX', 'A_Head_VelocityX', 'A_Head_DeltaZ', 'A_Head_VelocityZ']:
    groups = [group[feature].values for name, group in data.groupby('Scenario')]
    anova_results[feature] = f_oneway(*groups)

# Print ANOVA results
for feature, result in anova_results.items():
    print(f"Feature: {feature}, ANOVA F-value: {result.statistic}, p-value: {result.pvalue}")

# Prepare data for classification
X = data[['A_Head_DeltaX', 'A_Head_VelocityX', 'A_Head_DeltaZ', 'A_Head_VelocityZ']]
y = LabelEncoder().fit_transform(data['Scenario'])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Feature importance
feature_importances = clf.feature_importances_
features = X.columns

# Print feature importances
for feature, importance in zip(features, feature_importances):
    print(f"Feature: {feature}, Importance: {importance}")

# Visualize feature importance
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importances
})

fig = px.bar(importance_df, x='Feature', y='Importance', title='Feature Importance')
fig.show()

KeyError: 'A_Head_DeltaX'